**这是用来读取spei nc文件的测试代码**

In [127]:
# Step 1: 导入所需库
import xarray as xr
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point
import matplotlib.pyplot as plt

In [128]:
# Step 2: 读取 SPEI NetCDF 文件
spei_nc_path = '../data/raw/spei03.nc' 
ds = xr.open_dataset(spei_nc_path)
ds

<xarray.Dataset> Size: 2GB
Dimensions:  (lon: 720, lat: 360, time: 1476)
Coordinates:
  * lon      (lon) float64 6kB -179.8 -179.2 -178.8 -178.2 ... 178.8 179.2 179.8
  * lat      (lat) float64 3kB -89.75 -89.25 -88.75 -88.25 ... 88.75 89.25 89.75
  * time     (time) datetime64[ns] 12kB 1901-01-16 1901-02-15 ... 2023-12-16
Data variables:
    spei     (time, lat, lon) float32 2GB ...
    crs      int32 4B ...
Attributes: (12/15)
    conventions:  CF-1.11
    title:        Global 3-months SPEI, z-values, 0.5 degree
    version:      2.10.0
    id:           ./outputNcdf/spei03.nc
    summary:      Global dataset of the Standardized Precipitation-Evapotrans...
    keywords:     drought, climatology, SPEI, Standardized Precipitation-Evap...
    ...           ...
    software:     Created in R using the SPEI package (https://cran.r-project...
    call:         spei.nc(sca=i, inPre=./inputData/cru_ts4.08.1901.2023.pre.d...
    date:         Wed Jul 10 09:02:28 2024
    reference:    Beguería S., Vicente-Serrano S., Reig F., Latorre B. (2014)...
    reference2:   Vicente-Serrano S.M., Beguería S., López-Moreno J.I. (2010)...
    reference3:   Beguería S., Vicente-Serrano S., Angulo-Martínez M. (2010) ...

In [129]:
spei = ds['spei']  
print(spei)
lats = ds['lat'].values
lons = ds['lon'].values
times = pd.to_datetime(ds['time'].values)
print('纬度范围:', lats.min(), '-', lats.max())
print('经度范围:', lons.min(), '-', lons.max())
print('时间范围:', times.min(), '-', times.max())

<xarray.DataArray 'spei' (time: 1476, lat: 360, lon: 720)> Size: 2GB
[382579200 values with dtype=float32]
Coordinates:
  * lon      (lon) float64 6kB -179.8 -179.2 -178.8 -178.2 ... 178.8 179.2 179.8
  * lat      (lat) float64 3kB -89.75 -89.25 -88.75 -88.25 ... 88.75 89.25 89.75
  * time     (time) datetime64[ns] 12kB 1901-01-16 1901-02-15 ... 2023-12-16
Attributes:
    units:         1
    long_name:     Standardized Precipitation-Evapotranspiration Index
    grid_mapping:  crs
纬度范围: -89.75 - 89.75
经度范围: -179.75 - 179.75
时间范围: 1901-01-16 00:00:00 - 2023-12-16 00:00:00


In [130]:
# Step 3: 读取国家边界矢量文件
shapefile_path = '../data/raw/ne_50m_admin_0_countries/ne_50m_admin_0_countries.shp'  # 修改为你的实际路径
countries = gpd.read_file(shapefile_path).to_crs('EPSG:4326')
countries[['ISO_A3', 'geometry']].head()

# 提取国家名称和ISO_3代码
namelist = countries[['ADMIN', 'ISO_A3']].drop_duplicates().reset_index(drop=True)
namelist = namelist.rename(columns={'ADMIN': 'country_name', 'ISO_A3': 'iso3'})

# 输出table
ne_50m_admin_0_countries_namelist = namelist
display(ne_50m_admin_0_countries_namelist)

# 保存到processed文件夹
ne_50m_admin_0_countries_namelist.to_csv('../data/processed/ne_50m_admin_0_countries_namelist.csv', index=False)

,country_name,iso3
0,Zimbabwe,ZWE
1,Zambia,ZMB
2,Yemen,YEM
3,Vietnam,VNM
4,Venezuela,VEN
...,...,...
237,Afghanistan,AFG
238,Siachen Glacier,-99
239,Antarctica,ATA
240,Sint Maarten,SXM


In [131]:
# Step 4: 构建SPEI网格点GeoDataFrame
lon_grid, lat_grid = np.meshgrid(lons, lats)
points = [Point(lon, lat) for lon, lat in zip(lon_grid.flatten(), lat_grid.flatten())]
points_gdf = gpd.GeoDataFrame({'orig_idx': np.arange(len(points))}, geometry=points, crs='EPSG:4326')

In [132]:
# Step 5: 空间连接，将每个网格点分配到国家
points_gdf = gpd.sjoin(points_gdf, countries[['ADMIN','ISO_A3', 'geometry']], how='inner', predicate='within')
points_gdf = points_gdf.rename(columns={'ADMIN': 'country'})
points_gdf.head

<bound method NDFrame.head of         orig_idx                geometry  index_right     country ISO_A3
0              0  POINT (-179.75 -89.75)          239  Antarctica    ATA
1              1  POINT (-179.25 -89.75)          239  Antarctica    ATA
2              2  POINT (-178.75 -89.75)          239  Antarctica    ATA
3              3  POINT (-178.25 -89.75)          239  Antarctica    ATA
4              4  POINT (-177.75 -89.75)          239  Antarctica    ATA
...          ...                     ...          ...         ...    ...
249423    249423    POINT (-28.25 83.25)          181   Greenland    GRL
249424    249424    POINT (-27.75 83.25)          181   Greenland    GRL
249425    249425    POINT (-27.25 83.25)          181   Greenland    GRL
249426    249426    POINT (-26.75 83.25)          181   Greenland    GRL
249427    249427    POINT (-26.25 83.25)          181   Greenland    GRL

[85699 rows x 5 columns]>

In [133]:
# Step 6: 提取2019-2022每国每月SPEI均值
records = []
for t_idx, t in enumerate(times):
    if t.year < 2019 or t.year > 2022:
        continue
    spei_slice = spei.isel(time=t_idx).values.flatten()
    points_gdf['spei'] = spei_slice[points_gdf['orig_idx'].values]
    grouped = points_gdf.groupby(['country','ISO_A3'])['spei'].mean().reset_index()
    grouped['date'] = t
    records.append(grouped)
result = pd.concat(records, ignore_index=True)
result = result.dropna(subset=['country'])



现在得到了空间链接后的数据，接着检查数据情况

In [134]:
# 查看result的基本信息
print(result.info())

# 查看各字段的唯一值数量
print("country唯一值数量：", result['country'].nunique())
print("ISO_A3唯一值数量：", result['ISO_A3'].nunique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9120 entries, 0 to 9119
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   country  9120 non-null   object        
 1   ISO_A3   9120 non-null   object        
 2   spei     8976 non-null   float32       
 3   date     9120 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float32(1), object(2)
memory usage: 249.5+ KB
None
country唯一值数量： 190
ISO_A3唯一值数量： 185


In [135]:
# 1. shp文件所有国家/地区
all_admin = set(countries['ADMIN'].unique())
print(f"shp文件中ADMIN唯一值数量: {len(all_admin)}")

# 2. 空间连接后country的唯一值
joined_admin = set(result['country'].unique())
print(f"空间连接后country唯一值数量: {len(joined_admin)}")

# 3. 哪些shp里的国家没有出现在result里
missing_admin = all_admin - joined_admin
print(f"shp中但空间连接后缺失的国家/地区: {list(missing_admin)}")

# 4. result中ISO_A3为-99或缺失的情况
print("result中ISO_A3为-99的国家：", result.loc[result['ISO_A3'] == '-99', 'country'].unique())
print("result中ISO_A3缺失的数量：", result['ISO_A3'].isnull().sum())

# 5. 检查result中country和ISO_A3的对应关系
print("country和ISO_A3一对多的情况：")
print(result.groupby('ISO_A3')['country'].nunique().value_counts())

shp文件中ADMIN唯一值数量: 242
空间连接后country唯一值数量: 190
shp中但空间连接后缺失的国家/地区: ['Liechtenstein', 'Heard Island and McDonald Islands', 'Aruba', 'Montserrat', 'Grenada', 'Marshall Islands', 'Saint Lucia', 'Tonga', 'Guam', 'Seychelles', 'Hong Kong S.A.R.', 'British Indian Ocean Territory', 'São Tomé and Principe', 'Saint Martin', 'Pitcairn Islands', 'British Virgin Islands', 'Antigua and Barbuda', 'Guernsey', 'Sint Maarten', 'San Marino', 'Singapore', 'Malta', 'Tuvalu', 'Bermuda', 'Norfolk Island', 'Curaçao', 'Saint Barthelemy', 'Nauru', 'Bahrain', 'Monaco', 'Jersey', 'Vatican', 'Turks and Caicos Islands', 'Saint Kitts and Nevis', 'Andorra', 'Wallis and Futuna', 'Isle of Man', 'Indian Ocean Territories', 'Barbados', 'Federated States of Micronesia', 'Saint Helena', 'Ashmore and Cartier Islands', 'Anguilla', 'Saint Pierre and Miquelon', 'American Samoa', 'Niue', 'Palau', 'Maldives', 'Northern Mariana Islands', 'Macao S.A.R', 'Cook Islands', 'Dominica']
result中ISO_A3为-99的国家： ['France' 'Kosovo' 'Northern 

In [136]:
# 2. 检查空间连接后缺失的国家的多边形面积
missing_countries_gdf = countries[countries['ADMIN'].isin(missing_admin)].copy()
missing_countries_gdf['area'] = missing_countries_gdf['geometry'].to_crs('EPSG:3857').area / 1e6  # 单位：平方公里
print(missing_countries_gdf[['ADMIN', 'area']].sort_values('area'))

# 3. 检查是否有SPEI网格点落入这些国家（带序号输出）
for idx, admin in enumerate(missing_admin, 1):
    poly = missing_countries_gdf[missing_countries_gdf['ADMIN'] == admin].geometry.iloc[0]
    n_points = points_gdf.within(poly).sum()
    print(f"{idx}. {admin} 内的SPEI网格点数: {n_points}")


                                 ADMIN         area
5                              Vatican     1.278113
229        Ashmore and Cartier Islands     2.856467
241                             Tuvalu    10.503433
108                             Monaco    23.847567
196                        Macao S.A.R    26.595111
164                   Saint Barthelemy    27.142060
100                              Nauru    27.859625
240                       Sint Maarten    46.381684
20                    Pitcairn Islands    46.713122
228                     Norfolk Island    53.958729
163                       Saint Martin    54.358330
114                           Maldives    66.847959
95                        Cook Islands    69.953413
27                          Montserrat    87.193406
21                            Anguilla    92.520471
24                             Bermuda    92.584186
29                            Guernsey   114.037389
14                      American Samoa   128.520951
69          

下面代码解决country和ISO_A3一对多的情况，也就是：result中ISO_A3为-99的国家： ['France' 'Kosovo' 'Northern Cyprus' 'Norway' 'Siachen Glacier'
 'Somaliland']，其中Northern Cyprus不被联合国承认，所以没有iso-A3，我们这里直接删除；其他的手动匹配

In [137]:
# 直接从result中删除Northern Cyprus
result = result[result['country'] != 'Northern Cyprus'].copy()

In [138]:
# 1. 构建手动映射表（示例，需根据实际情况补充）
iso3_manual_map = {
    'France': 'FRA',
    'Kosovo': 'XKX',
    'Norway': 'NOR',
    'Siachen Glacier': 'SIA',
    'Somaliland': 'SOL'
}

# 2. 替换spei_clean中的ISO_A3
def fix_iso3(row):
    if row['ISO_A3'] == '-99':
        return iso3_manual_map.get(row['country'], None)
    else:
        return row['ISO_A3']

result = result[result['country'] != 'Northern Cyprus'].copy()
result['ISO_A3_fixed'] = result.apply(fix_iso3, axis=1)

# 3. 检查修正结果
print(result[result['ISO_A3'] == '-99'][['country', 'ISO_A3', 'ISO_A3_fixed']])

# 4. 用修正后的ISO3替换原有列
result['ISO_A3'] = result['ISO_A3_fixed']
result = result.drop(columns=['ISO_A3_fixed'])


              country ISO_A3 ISO_A3_fixed
57             France    -99          FRA
89             Kosovo    -99          XKX
124            Norway    -99          NOR
146   Siachen Glacier    -99          SIA
152        Somaliland    -99          SOL
...               ...    ...          ...
8987           France    -99          FRA
9019           Kosovo    -99          XKX
9054           Norway    -99          NOR
9076  Siachen Glacier    -99          SIA
9082       Somaliland    -99          SOL

[240 rows x 3 columns]


In [139]:
# 检查新的result，统计国家和ISO_A3的唯一数量
print("去除Northern Cyprus后，result中country唯一值数量：", result['country'].nunique())
print("去除Northern Cyprus后，result中ISO_A3唯一值数量：", result['ISO_A3'].nunique())

去除Northern Cyprus后，result中country唯一值数量： 189
去除Northern Cyprus后，result中ISO_A3唯一值数量： 189


检查完成：
1. 上面52个没有连接上的国家，是因为面积太小，没有spei网格点落入；
2. 修正了5个ISO-3错误的国家代码（-99->real ISO-A3），删除了北塞浦路斯这个不被联合国承认的国家
3. 242-52-1 = 189
3. 目前的result是都有至少一个网格点落入国界的，但是由于spei可能存在空值，所以需要再次检查。

In [140]:
# 检查整体缺失
print("总行数：", len(result))
print("每列缺失值数量：\n", result.isnull().sum())

# 检查哪些国家有缺失
missing_country = result[result['spei'].isnull()]
print("有空值的国家数量：", missing_country['country'].nunique())
print("有空值的国家代码：", missing_country['country'].unique())

总行数： 9072
每列缺失值数量：
 country      0
ISO_A3       0
spei       144
date         0
dtype: int64
有空值的国家数量： 3
有空值的国家代码： ['Antarctica' 'Cayman Islands' 'Kiribati']


In [141]:
# 检查Kiribati
country_poly = countries[countries['ADMIN'] == 'Kiribati'].geometry.iloc[0]
points_in_country = points_gdf[points_gdf.within(country_poly)]
print(f"Kiribati内的网格点数: {len(points_in_country)}")

# 找到该点的经纬度
pt = points_in_country.geometry.iloc[0]
lon, lat = pt.x, pt.y

# 找到最近的SPEI网格索引
lat_idx = np.argmin(np.abs(lats - lat))
lon_idx = np.argmin(np.abs(lons - lon))

# 检查所有时间的SPEI值
spei_values = spei[:, lat_idx, lon_idx].values
print("该点所有时间的SPEI值是否全为NaN：", np.all(np.isnan(spei_values)))
print("部分SPEI值：", spei_values[:10])

Kiribati内的网格点数: 1
该点所有时间的SPEI值是否全为NaN： True
部分SPEI值： [nan nan nan nan nan nan nan nan nan nan]


In [142]:
# 检查Cayman Islands
country_poly_CI = countries[countries['ADMIN'] == 'Cayman Islands'].geometry.iloc[0]
points_in_country_CI = points_gdf[points_gdf.within(country_poly_CI)]
print(f"Cayman Islands内的网格点数: {len(points_in_country_CI)}")

# 找到该点的经纬度
pt_CI = points_in_country_CI.geometry.iloc[0]
lon_CI, lat_CI = pt_CI.x, pt_CI.y

# 找到最近的SPEI网格索引
lat_idx_CI = np.argmin(np.abs(lats - lat_CI))
lon_idx_CI = np.argmin(np.abs(lons - lon_CI))

# 检查所有时间的SPEI值
spei_values_CI = spei[:, lat_idx_CI, lon_idx_CI].values
print("该点所有时间的SPEI值是否全为NaN：", np.all(np.isnan(spei_values_CI)))
print("部分SPEI值：", spei_values_CI[:10])


Cayman Islands内的网格点数: 1
该点所有时间的SPEI值是否全为NaN： True
部分SPEI值： [nan nan nan nan nan nan nan nan nan nan]


In [143]:
# 检查Antarctica
country_poly_A = countries[countries['ADMIN'] == 'Antarctica'].geometry.iloc[0]
points_in_country_A = points_gdf[points_gdf.within(country_poly_A)]
print(f"Antarctica内的网格点数: {len(points_in_country_A)}")

# 找到该点的经纬度
pt_A = points_in_country_A.geometry.iloc[0]
lon_A, lat_A = pt_A.x, pt_A.y

# 找到最近的SPEI网格索引
lat_idx_A = np.argmin(np.abs(lats - lat_A))
lon_idx_A = np.argmin(np.abs(lons - lon_A))

# 检查所有时间的SPEI值
spei_values_A = spei[:, lat_idx_A, lon_idx_A].values
print("该点所有时间的SPEI值是否全为NaN：", np.all(np.isnan(spei_values_A)))
print("部分SPEI值：", spei_values_A[:10])


Antarctica内的网格点数: 24165
该点所有时间的SPEI值是否全为NaN： True
部分SPEI值： [nan nan nan nan nan nan nan nan nan nan]


所以检查的结果是，三个地方的都有spei网格点，但是spei的值确实都是Nan

In [144]:
# 删除有缺失的三个国家
drop_countries = ['Antarctica', 'Cayman Islands', 'Kiribati']
spei_clean = result[~result['country'].isin(drop_countries)].copy()

# 检查是否还有缺失
print("每列缺失值数量：\n", spei_clean.isnull().sum())
print("总行数：", len(spei_clean))

每列缺失值数量：
 country    0
ISO_A3     0
spei       0
date       0
dtype: int64
总行数： 8928


In [145]:
spei_clean

,country,ISO_A3,spei,date
0,Afghanistan,AFG,-0.101823,2019-01-16
1,Aland,ALA,-0.584555,2019-01-16
2,Albania,ALB,-0.229678,2019-01-16
3,Algeria,DZA,-1.081419,2019-01-16
4,Angola,AGO,0.090749,2019-01-16
...,...,...,...,...
9115,Western Sahara,ESH,-1.458937,2022-12-16
9116,Yemen,YEM,-1.207492,2022-12-16
9117,Zambia,ZMB,0.084212,2022-12-16
9118,Zimbabwe,ZWE,-0.164138,2022-12-16


现在完成了数据的清理，得到的spei_country_month_2019_2022_cleaned.csv是没有缺失值的，ISO3正确且完整的

In [146]:
# 最终数据检查

print("时间范围：", spei_clean['date'].min(), "到", spei_clean['date'].max())
print("spei列是否有空值：", spei_clean['spei'].isnull().sum() == 0)
print("ISO_A3列是否有空值：", spei_clean['ISO_A3'].isnull().sum() == 0)
print("ISO_A3是否还有-99：", (spei_clean['ISO_A3'] == '-99').sum() == 0)
print("国家数量：", spei_clean['country'].nunique())
print("ISO_A3数量：", spei_clean['ISO_A3'].nunique())

# 每个国家的spei数量
spei_count = spei_clean.groupby('country')['spei'].count()
print("每个国家的spei数量是否都是48：", (spei_count == 48).all())
print("spei数量不是48的国家：")
print(spei_count[spei_count != 48])

时间范围： 2019-01-16 00:00:00 到 2022-12-16 00:00:00
spei列是否有空值： True
ISO_A3列是否有空值： True
ISO_A3是否还有-99： True
国家数量： 186
ISO_A3数量： 186
每个国家的spei数量是否都是48： True
spei数量不是48的国家：
Series([], Name: spei, dtype: int64)


In [147]:
# 5. 保存
spei_clean.to_csv('../data/processed/spei_country_month_2019_2022_cleaned.csv', index=False)
print("最终面板已保存，无空值，每国48期，ISO3完整。保存为 spei_country_month_2019_2022_cleaned.csv")

最终面板已保存，无空值，每国48期，ISO3完整。保存为 spei_country_month_2019_2022_cleaned.csv
